# Lab 9
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marts-dev/geog510-test/blob/main/lab9.ipynb)


In [ ]:
# %pip install -U leafmap
# %pip install -U rasterio
# %pip install -U geopandas

In [2]:
import leafmap
import rasterio
from rasterio.warp import transform_bounds
import geopandas as gpd

## Question 1: Creating an Interactive Map
Create an interacctive map with search functionality that allows users to search for places and zoom to them. Disable the draw control on the map.

In [ ]:
m = leafmap.Map(draw_control=False, search_control=True)
m.search_control.zoom = 15
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Question 2: Adding Map Legends
1. Add ESA World Cover WMS tile layer to the map.
 - URL: `https://services.terrascope.be/wms/v2?`
 - Layer name: `WORLDCOVER_2021_MAP`
2. Add a legend to the map using the leafmap built-in `ESA_WorldCover` legend.

In [4]:
m = leafmap.Map(draw_control=False, search_control=True, zoom=2)
m.search_control.zoom = 15
url = "https://services.terrascope.be/wms/v2?"
layer_name = "WORLDCOVER_2021_MAP"
m.add_wms_layer(
  url=url,
  layers=layer_name,
  name=layer_name,
  format="image/png",
  show=True,
)
m.add_legend(title="ESA Land Cover Type", builtin_legend="ESA_WorldCover")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Question 3: Creating Marker Clusters
1. Create a marker cluster visualization from a GeoJSON file of building centroids:
 - URL: [opengeos/datasets](https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson)
 - Hint: Read the GeoJSON file using GeoPandas and add "latitude" and "longitude" columns to the GeoDataFrame.
2. Create circle markers for each building centroid using the `Map.add_circle_markers_from_xy()` method with the following styling:
 - Radius: 5
 - Outline color: "red"
 - Fill color: "yellow"
 - Fill opacity: 0.8

In [5]:
url = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"

gdf = gpd.read_file(url)

In [6]:
gdf.explore()

In [7]:
gdf.head()

,Class,Confidence,Shape_Leng,Shape_Area,geometry
0,1,28.905460,68.457069,292.568026,POINT (-117.60109 47.65499)
1,1,99.420196,97.152785,556.702899,POINT (-117.59953 47.65533)
2,1,95.450807,90.916993,492.940128,POINT (-117.59991 47.65514)
3,1,91.446453,85.645123,453.842109,POINT (-117.59953 47.65575)
4,1,90.172392,78.057638,380.403649,POINT (-117.59989 47.65534)


In [8]:
gdf['longitude'] = gdf['geometry'].x
gdf['latitude'] = gdf['geometry'].y

In [9]:
gdf.head()

,Class,Confidence,Shape_Leng,Shape_Area,geometry,longitude,latitude
0,1,28.905460,68.457069,292.568026,POINT (-117.60109 47.65499),-117.601092,47.654993
1,1,99.420196,97.152785,556.702899,POINT (-117.59953 47.65533),-117.599525,47.655326
2,1,95.450807,90.916993,492.940128,POINT (-117.59991 47.65514),-117.599910,47.655143
3,1,91.446453,85.645123,453.842109,POINT (-117.59953 47.65575),-117.599532,47.655747
4,1,90.172392,78.057638,380.403649,POINT (-117.59989 47.65534),-117.599892,47.655336


In [10]:
m = leafmap.Map(draw_control=False, search_control=True, zoom=2)
layer = m.add_circle_markers_from_xy(
    gdf,
    x="longitude",
    y="latitude",
    radius=5,
    color="red",
    fill_color="yellow",
    fill_opacity=0.8,
    layer_name="Building Centroids",
    zoom=15,
)

m.zoom_to_bounds(gdf.total_bounds)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Question 4: Visualizing Vector Data
1. Visualize the building polygons GeoJSON file and style it with:
 - Outline color: “red”
 - No fill color
 - URL: [opengeos/datasets](https://github.com/opengeos/datasets/releases/download/places/wa_overture_buildings.geojson)

2. Visualize the road polylines GeoJSON file and style it with:
 - Line color: “red”
 - Line width: 2
 - URL: [opengeos/datasets](https://github.com/opengeos/datasets/releases/download/places/las_vegas_roads.geojson)

3. Create a choropleth map of county areas in the US:
 - URL: [opengeos/datasets](https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson)
 - Column: `CENSUSAREA`

In [11]:
m = leafmap.Map(basemap="Google Satellite")
url = "https://github.com/opengeos/datasets/releases/download/places/wa_overture_buildings.geojson"
m.add_vector(url, layer_name="Overture Buildings", style={"color": "red", "fill_color": "none"}, zoom_to_layer=True)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [12]:
m = leafmap.Map(basemap="Google Satellite")
url = "https://github.com/opengeos/datasets/releases/download/places/las_vegas_roads.geojson"
m.add_vector(url, layer_name="Las Vegas Roads", style={"color": "red", "weight": 2}, zoom_to_layer=True)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [13]:
m = leafmap.Map()
url = "https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson"
gdf = gpd.read_file(url)
m.add_data(
  gdf,
  column="CENSUSAREA",
  cmap="Blues",
  legend_title="Census Area",
)

m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Question 5: Creating a Split Map
Create a split map to compare imagery of Libya before and after the 2023 flood event:
 - Pre-event imagery: [opengeos/datasets](https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-07-01.tif)
 - Post-event imagery: [opengeos/datasets](https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-09-13.tif)

In [14]:
m = leafmap.Map(zoom=2)
left_url = 'https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-07-01.tif'
right_url = 'https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-09-13.tif'

with rasterio.open(left_url) as src:
    bounds = src.bounds
    src_crs = src.crs

new_bounds = transform_bounds(
    src_crs, "EPSG:4326", *bounds
)
m.split_map(
    left_layer=left_url,
    right_layer=right_url,
    left_label="Before Libya Flood 2023",
    right_label="After Libya Flood 2023",
)
m.zoom_to_bounds(new_bounds)

m


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…